# Project 

In [1]:
import pandas as pd
import re
import seaborn as sns
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords 
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import math
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer

# The following line is needed to show plots inline in notebooks
%matplotlib inline 

[nltk_data] Downloading package wordnet to /home/bhasingu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bhasingu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.head()

,Unnamed: 0,claim,claimant,date,id,label,related_articles
0,0,A line from George Orwell's novel 1984 predict...,NaN,17/07/2017,0,0,"[122094, 122580, 130685, 134765]"
1,1,Maine legislature candidate Leslie Gibson insu...,NaN,17/03/2018,1,2,"[106868, 127320, 128060]"
2,2,A 17-year-old girl named Alyssa Carson is bein...,NaN,18/07/2018,4,1,"[132130, 132132, 149722]"
3,3,In 1988 author Roald Dahl penned an open lette...,NaN,04/02/2019,5,2,"[123254, 123418, 127464]"
4,4,"When it comes to fighting terrorism, ""Another ...",Hillary Clinton,22/03/2016,6,2,"[41099, 89899, 72543, 82644, 95344, 88361]"


In [4]:
def str2array(value):
    str_list = re.findall(r'\d+', value)
    int_list = list(map(int, str_list))
    article_array = np.array(int_list)
    return article_array

In [5]:
#related_article_array = str2array(data.iloc[:, 6])
data['article_array'] = data['related_articles'].apply(str2array)

In [6]:
# print(np.shape(data.iloc[0, -1]))
data['related_article_text'] = [list() for x in range(len(data.index))]

In [7]:
# print(len(data.iloc[1,7]))
import os
os.chdir(os.getcwd() + '/train_articles')

# print(os.getcwd())

In [8]:
# Importing related article text to 'data' dataframe 
for i in range(len(data)):
    for j in range(len(data.iloc[i, 7])):
        f=open(str(data.iloc[i, 7][j])+'.txt', "r")
        if f.mode == 'r':
            contents =f.read()
            data.related_article_text[i].append(contents)
#             data.at[i, 'related_article_text'] = data.iloc[i, -1]+''+contents
        f.close()

In [9]:
# data.related_article_text[0][2]
# data

In [10]:
data['related_article_text_clean'] = [list() for x in range(len(data.index))]
# data
# len(data.related_article_text[0])

In [11]:
%%time

stemmer = SnowballStemmer("english")
from IPython.display import clear_output, display
tokenizer = RegexpTokenizer(r'\w+') #Splits up the words
stop_words = set(stopwords.words('english'))
related_article_text_clean = [] 
i=0
for i in range(len(data)):
    for j in range(len(data.related_article_text[i])):
#     snip = re.sub(r'\d+', '', text) #remove numeric
        text = data.related_article_text[i][j]
        words = tokenizer.tokenize(text)
        words = [w.lower() for w in words] #lowercase  
        words = [w for w in words if not w in stop_words] #remove stop words
        words = [stemmer.stem(w) for w in words] #stemming
    
        sentence = ' '.join(word for word in words)
        data.related_article_text_clean[i].append(sentence)
    i+=1
    clear_output(wait=True)
    print(str(round((i/len(data))*100, 2))+'%')

100.0%
CPU times: user 16min 25s, sys: 11.3 s, total: 16min 36s
Wall time: 16min 37s


In [12]:
# data['related_article_text_clean'] = ''
# data['related_article_text_clean'] = pd.Series(related_article_text_clean)
# data

In [13]:
# print(type(related_article_text_clean[0]))
# data.drop(data.tail(1).index,inplace=True) # drop last n rows

In [14]:
%%time

stemmer = SnowballStemmer("english")
from IPython.display import clear_output, display
tokenizer = RegexpTokenizer(r'\w+') #Splits up the words
stop_words = set(stopwords.words('english'))
claim_clean = [] 
i=0
for text in data['claim']:
    snip = re.sub(r'\d+', '', text) #remove numeric
    words = tokenizer.tokenize(text)
    words = [w.lower() for w in words] #lowercase  
    words = [w for w in words if not w in stop_words] #remove stop words
    words = [stemmer.stem(w) for w in words] #stemming
    
    sentence = ' '.join(word for word in words)
    claim_clean.append(sentence)
    i+=1
    clear_output(wait=True)
    print(str(round((i/len(data))*100, 2))+'%')

100.0%
CPU times: user 26.7 s, sys: 3.46 s, total: 30.1 s
Wall time: 29.6 s


In [15]:
data['claim_clean'] = ''
data['claim_clean'] = pd.Series(claim_clean)
# related_article_text_clean

In [16]:
# print(type(related_article_text_clean[0]))
# data.drop(data.tail(1).index,inplace=True) # drop last n rows

In [17]:
# data

In [18]:
# data.top_words_article[0]

In [19]:
data['words_claim'] = ''
# data['freq_claim'] = ''
for i in range(len(data)):
    key_list = []
    value_list = []
    str1 = data.claim_clean[i]
    words = str1.split(' ')
    c = Counter(words)
#     com = c.most_common()
#     comm = dict(com[0:20])
    j=0
    for keys in dict(c):
        key_list.append(keys)
#         value_list.append(comm[keys])
        j+=1
    data.at[i, 'words_claim'] = key_list
#     data.at[i, 'freq_claim'] = value_list
    clear_output(wait=True)
    print(str(round((i/len(data))*100, 2))+'%')

99.99%


In [20]:
from collections import Counter

data['top_words_article'] = ''
data['freq_article'] = ''

for i in range(len(data)):
    str1 = ''
    for j in range(len(data.related_article_text_clean[i])):
        str1 += data.related_article_text_clean[i][j]
    key_list = []
    value_list = []
    words = str1.split(' ')
    c = Counter(words)
    com = c.most_common()
    comm = dict(com[0:len(data.words_claim[i])])
    for keys in dict(comm):
        key_list.append(keys)
        value_list.append(comm[keys])
    data.at[i, 'top_words_article'] = key_list
    data.at[i, 'freq_article'] = value_list
    clear_output(wait=True)
    print(str(round((i/len(data))*100, 2))+'%')
#     data.freq[i] = 



99.99%


In [21]:
# data

In [22]:
data['weighted_sum'] = ''
for i in range(len(data)):
    w_sum = 0
    j = 0
    for word in (data.words_claim[i]):
        if word in data.top_words_article[i]:
            w_sum += (data.freq_article[i][data.top_words_article[i].index(word)])/len(data.words_claim[i])
    data.at[i, 'weighted_sum'] = w_sum

In [23]:
# data

In [24]:
# print(data.top_words_article[1])
# print(data.claim[0])
# print(data.freq_article[1])
# print(data.weighted_sum[1])
# print(data.claim[0])
# print(data.related_article_text_clean[0])

In [25]:
# Counting number of words in claim
data['num_words_claim'] = ''
for i in range(len(data)):
    str1 = data.claim[i]
    words = str1.split(' ')
#     c = Counter(words)
    data.at[i, 'num_words_claim'] = len(words)

In [26]:
# Counting number of characters in claim
data['num_char_claim'] = ''
for i in range(len(data)):
    data.at[i, 'num_char_claim'] = len(data.claim[i])

In [27]:
data['num_stop_claim'] = ''
for i in range(len(data)):
    str1 = data.claim[i]
    words = str1.split()
    counter = 0
    for w in words:
        if w in stop_words:
            counter += 1
    data.at[i, 'num_stop_claim'] = counter

In [28]:
data['num_special_claim'] = ''
for i in range(len(data)):
#     str1 = data.claim[i]
#     words = str1.split()
    counter = 0
    for c in data.claim[i]:
        if c in '[@_!#$%^&*()<>?/\|}{~:;+=-`]':
            counter += 1
    data.at[i, 'num_special_claim'] = counter

In [29]:
# data

## Data Cleaning

In [46]:
feature_df = data.copy()
del feature_df['Unnamed: 0']
del feature_df['id']
del feature_df['claim']
del feature_df['related_articles']
del feature_df['article_array']
del feature_df['related_article_text']
del feature_df['related_article_text_clean']
del feature_df['claim_clean']
del feature_df['words_claim']
del feature_df['top_words_article']
del feature_df['freq_article']
# feature_df

,claimant,date,label,weighted_sum,num_words_claim,num_char_claim,num_stop_claim,num_special_claim
0,NaN,17/07/2017,0,4.5,12,73,3,0
1,NaN,17/03/2018,2,4.71429,15,117,1,0
2,NaN,18/07/2018,1,8.54545,15,87,5,2
3,NaN,04/02/2019,2,13.5833,18,113,5,0
4,Hillary Clinton,22/03/2016,2,10.0833,22,129,9,0
5,Leonidas Raptakis,11/02/2014,2,11.8889,25,162,7,1
6,Jim Webb,19/11/2014,1,1.625,18,97,9,0
7,NaN,18/07/2013,0,7.28571,9,56,2,0
8,Robin Vos,22/08/2013,1,10.6154,23,135,8,0
9,Nick Schroer,17/10/2017,1,3.13333,26,141,11,0


### Claimant

In [47]:
# There are 4962 missing values in claimant
feature_df[feature_df['claimant'].isna()].shape[0]

4962

In [48]:
claimant_count = feature_df['claimant'].value_counts()
claimant_count

Donald Trump                   1233
Bloggers                        372
Barack Obama                    234
Hillary Clinton                 220
Viral image                     127
Facebook posts                  108
Ted Cruz                        106
Various websites                106
Bernie Sanders                  101
Marco Rubio                      97
Scott Walker                     90
John McCain                      88
Rick Perry                       77
Rick Scott                       73
Chain email                      71
Facebook user                    71
multiple sources                 70
Mike Pence                       60
Jeb Bush                         58
Paul Ryan                        51
Multiple sources                 50
Vladimir Putin                   44
Greg Abbott                      43
Charlie Crist                    43
Chris Christie                   41
Mitt Romney                      39
Tim Kaine                        39
Ron Johnson                 

In [49]:
# Group together all counts less than 100 in to Others
value_mask = feature_df.claimant.isin(claimant_count.index[claimant_count < 100]) 
feature_df.loc[value_mask,'claimant'] = "Other_claimant"
feature_df.claimant.value_counts()

Other_claimant      7986
Donald Trump        1233
Bloggers             372
Barack Obama         234
Hillary Clinton      220
Viral image          127
Facebook posts       108
Ted Cruz             106
Various websites     106
Bernie Sanders       101
Name: claimant, dtype: int64

In [50]:
# Of the claimant missing, how many are false-0, partly true-1, true-2?
missing_data = feature_df[feature_df['claimant'].isna()]
missing_data.label.value_counts(normalize=True)
# roughly 87% of claims with missing claimants are not completely true

0    0.611447
1    0.259371
2    0.129182
Name: label, dtype: float64

### Date

In [51]:
# 5% of data happen on 2 separate days, November 6th 2017 and October 23rd 2016
date_count = feature_df['date'].value_counts()
# date_count

06/11/2017    407
23/10/2016    339
07/07/2016     97
26/09/2018     59
04/10/2016     35
25/02/2019     33
30/01/2019     31
07/03/2019     30
09/10/2016     29
19/10/2016     29
20/02/2019     29
22/10/2018     28
30/01/2018     27
07/02/2019     26
14/09/2018     26
05/02/2019     25
10/04/2019     25
14/02/2019     25
12/12/2018     25
08/05/2018     25
13/03/2018     24
04/04/2019     24
26/10/2018     24
12/03/2019     23
07/09/2016     23
10/10/2018     23
25/09/2018     23
11/02/2019     23
23/08/2017     23
27/07/2016     23
             ... 
13/06/2008      1
28/12/2014      1
29/06/2011      1
14/12/2015      1
05/01/2011      1
21/03/2008      1
02/05/2008      1
17/02/2013      1
01/06/2011      1
16/02/2010      1
25/02/2017      1
20/02/2016      1
02/06/2013      1
31/01/2008      1
06/08/2016      1
17/02/2009      1
02/09/2009      1
23/12/2010      1
05/03/2012      1
04/04/2011      1
01/07/2007      1
08/01/2015      1
08/06/2015      1
13/08/2010      1
23/05/2014

In [52]:
# On average, about 5 claims happen a day
feature_df.date.value_counts().mean()

5.152368333885392

In [53]:
# Group together all counts less than average in to Others
value_mask = feature_df.date.isin(date_count.index[date_count < 30]) 
feature_df.loc[value_mask,'date'] = "Other_date"
feature_df.date.value_counts()

Other_date    14524
06/11/2017      407
23/10/2016      339
07/07/2016       97
26/09/2018       59
04/10/2016       35
25/02/2019       33
30/01/2019       31
07/03/2019       30
Name: date, dtype: int64

Cleaning columns

In [54]:
feature_df

,claimant,date,label,weighted_sum,num_words_claim,num_char_claim,num_stop_claim,num_special_claim
0,NaN,Other_date,0,4.5,12,73,3,0
1,NaN,Other_date,2,4.71429,15,117,1,0
2,NaN,Other_date,1,8.54545,15,87,5,2
3,NaN,Other_date,2,13.5833,18,113,5,0
4,Hillary Clinton,Other_date,2,10.0833,22,129,9,0
5,Other_claimant,Other_date,2,11.8889,25,162,7,1
6,Other_claimant,Other_date,1,1.625,18,97,9,0
7,NaN,Other_date,0,7.28571,9,56,2,0
8,Other_claimant,Other_date,1,10.6154,23,135,8,0
9,Other_claimant,Other_date,1,3.13333,26,141,11,0


## Encoding

### Claimant Encode

In [55]:
# del feature_df['claim']
# feature_df

,claimant,date,label,weighted_sum,num_words_claim,num_char_claim,num_stop_claim,num_special_claim
0,NaN,Other_date,0,4.5,12,73,3,0
1,NaN,Other_date,2,4.71429,15,117,1,0
2,NaN,Other_date,1,8.54545,15,87,5,2
3,NaN,Other_date,2,13.5833,18,113,5,0
4,Hillary Clinton,Other_date,2,10.0833,22,129,9,0
5,Other_claimant,Other_date,2,11.8889,25,162,7,1
6,Other_claimant,Other_date,1,1.625,18,97,9,0
7,NaN,Other_date,0,7.28571,9,56,2,0
8,Other_claimant,Other_date,1,10.6154,23,135,8,0
9,Other_claimant,Other_date,1,3.13333,26,141,11,0


In [56]:
# Label encoding
# data['claimant']=data['claimant'].astype('category')
# data['claimant_cat']=data['claimant'].cat.codes
# claimant_labels = dict(enumerate(data['claimant'].cat.categories))
# claimant_labels

In [57]:
# One hot encoding

# feature_df = pd.get_dummies(feature_df['claimant']).copy()
claimant_df = pd.get_dummies(feature_df['claimant']).copy()
# claimant_df

,Barack Obama,Bernie Sanders,Bloggers,Donald Trump,Facebook posts,Hillary Clinton,Other_claimant,Ted Cruz,Various websites,Viral image
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0
6,0,0,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0
9,0,0,0,0,0,0,1,0,0,0


In [58]:
feature_df = pd.concat([feature_df.copy(), claimant_df], axis=1, sort=False)

In [59]:
del feature_df['claimant']
# feature_df

,date,label,weighted_sum,num_words_claim,num_char_claim,num_stop_claim,num_special_claim,Barack Obama,Bernie Sanders,Bloggers,Donald Trump,Facebook posts,Hillary Clinton,Other_claimant,Ted Cruz,Various websites,Viral image
0,Other_date,0,4.5,12,73,3,0,0,0,0,0,0,0,0,0,0,0
1,Other_date,2,4.71429,15,117,1,0,0,0,0,0,0,0,0,0,0,0
2,Other_date,1,8.54545,15,87,5,2,0,0,0,0,0,0,0,0,0,0
3,Other_date,2,13.5833,18,113,5,0,0,0,0,0,0,0,0,0,0,0
4,Other_date,2,10.0833,22,129,9,0,0,0,0,0,0,1,0,0,0,0
5,Other_date,2,11.8889,25,162,7,1,0,0,0,0,0,0,1,0,0,0
6,Other_date,1,1.625,18,97,9,0,0,0,0,0,0,0,1,0,0,0
7,Other_date,0,7.28571,9,56,2,0,0,0,0,0,0,0,0,0,0,0
8,Other_date,1,10.6154,23,135,8,0,0,0,0,0,0,0,1,0,0,0
9,Other_date,1,3.13333,26,141,11,0,0,0,0,0,0,0,1,0,0,0


In [60]:
date_df = pd.get_dummies(feature_df['date']).copy()
# date_df

,04/10/2016,06/11/2017,07/03/2019,07/07/2016,23/10/2016,25/02/2019,26/09/2018,30/01/2019,Other_date
0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,1
7,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,0,1


In [61]:
feature_df = pd.concat([feature_df.copy(), date_df], axis=1, sort=False)
del feature_df['date']
# feature_df

,label,weighted_sum,num_words_claim,num_char_claim,num_stop_claim,num_special_claim,Barack Obama,Bernie Sanders,Bloggers,Donald Trump,...,Viral image,04/10/2016,06/11/2017,07/03/2019,07/07/2016,23/10/2016,25/02/2019,26/09/2018,30/01/2019,Other_date
0,0,4.5,12,73,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,4.71429,15,117,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,8.54545,15,87,5,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2,13.5833,18,113,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2,10.0833,22,129,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,2,11.8889,25,162,7,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,1,1.625,18,97,9,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,0,7.28571,9,56,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,1,10.6154,23,135,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,1,3.13333,26,141,11,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### Cleaned Data Frame

##### Split the test and train data

In [310]:
from sklearn.preprocessing import StandardScaler
X = np.array(feature_df.drop(['label'], axis=1).copy())
y = np.array(feature_df['label'].copy())

#Standardize X
scaler = StandardScaler()

# Split test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [311]:
# # from sklearn.datasets import load_digits
# from sklearn.decomposition import TruncatedSVD
# # float_formatter = lambda x: "%.2f" % x

# svd = TruncatedSVD()
# X_train = svd.fit_transform(X_train)
# X_test = svd.transform(X_test)

In [312]:
# explained_variance = svd.explained_variance_ratio_
# explained_variance

In [313]:
from sklearn.decomposition import PCA

pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [314]:
explained_variance = pca.explained_variance_ratio_
explained_variance

array([1.33932663e-01, 8.36583254e-02, 5.65296139e-02, 4.66939287e-02,
       4.40803930e-02, 4.36883748e-02, 4.29929754e-02, 4.29765174e-02,
       4.25397107e-02, 4.22059632e-02, 4.21080103e-02, 4.20281636e-02,
       4.18605278e-02, 4.17687128e-02, 4.17329800e-02, 4.12660643e-02,
       4.04125821e-02, 3.97076211e-02, 3.74624778e-02, 3.34409694e-02,
       1.68213522e-02, 1.92797660e-03, 1.64096279e-04, 3.53983667e-32])

In [315]:
pca = PCA(n_components=23)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

##### Neural Network

In [316]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from numpy import linalg as LA
from numpy.linalg import inv

def OneHot(train_Target, test_Target):
    new_train = np.zeros((train_Target.shape[0], 3))
    new_test = np.zeros((test_Target.shape[0], 3))
    for item in range(0,len(train_Target)):
        new_train[item][train_Target[item]] = 1
    for item in range(0, len(test_Target)):
        new_test[item][test_Target[item]] = 1
    return new_train, new_test

In [317]:
new_y_train, new_y_test = OneHot(y_train, y_test)

In [318]:
def multilayer_perceptron(x, weights, biases, keep_prob):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, keep_prob)
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

In [319]:
n_hidden_1 = 38
n_input = X_train.shape[1]
n_classes = 3

weights = {
    'h1': tf.Variable(tf.random.normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random.normal([n_hidden_1, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random.normal([n_hidden_1])),
    'out': tf.Variable(tf.random.normal([n_classes]))
}

keep_prob = tf.placeholder("float")

In [320]:
training_epochs = 5000
display_step = 1000
batch_size = 32

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [321]:
predictions = multilayer_perceptron(x, weights, biases, keep_prob)

In [322]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))

In [323]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)

In [324]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(X_train) / batch_size)
        x_batches = np.array_split(X_train, total_batch)
        y_batches = np.array_split(new_y_train, total_batch)
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: 0.8
                            })
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: X_test, y: new_y_test, keep_prob: 1.0}))

Epoch: 0001 cost= 10.182610966
Epoch: 1001 cost= 0.876239611
Epoch: 2001 cost= 0.871680035
Epoch: 3001 cost= 0.866084637
Epoch: 4001 cost= 0.863705683
Epoch: 5001 cost= 0.861942977
Epoch: 6001 cost= 0.862864936
Epoch: 7001 cost= 0.863713596
Epoch: 8001 cost= 0.862833450
Epoch: 9001 cost= 0.860781540
Optimization Finished!
Accuracy: 0.58924365
